In [1]:
import pandas as pd
import plotly.express as px

# Carregando os dados

In [2]:
data = pd.read_csv("data_prep", index_col = 0)
data

,Time,Mean Power D-2,Max Power D-2,Mean Power D-1,Max Power D-1,Mean Temp Last 2 Days,Month Id,Power Generated
0,00:00:00,738.15,2081,746.68,2085,13.34,1.0,0.0
1,00:00:00,738.15,2081,746.68,2085,13.34,1.0,0.0
2,00:05:00,738.15,2081,746.68,2085,13.34,1.0,0.0
3,00:10:00,738.15,2081,746.68,2085,13.34,1.0,0.0
4,00:15:00,738.15,2081,746.68,2085,13.34,1.0,0.0
...,...,...,...,...,...,...,...,...
98067,23:30:00,526.32,1393,534.41,1411,14.88,12.0,0.0
98068,23:35:00,526.32,1393,534.41,1411,14.88,12.0,0.0
98069,23:40:00,526.32,1393,534.41,1411,14.88,12.0,0.0
98070,23:45:00,526.32,1393,534.41,1411,14.88,12.0,0.0


In [17]:
weather = pd.read_csv("processed_porto_conds", index_col = 0)
weather.index = pd.to_datetime(weather.index)
weather

,Conditions
2019-01-11 00:00:00,Fair
2019-01-11 00:05:00,Fair
2019-01-11 00:10:00,Fair
2019-01-11 00:15:00,Fair
2019-01-11 00:20:00,Fair
...,...
2019-12-29 23:35:00,Fair
2019-12-29 23:40:00,Fair
2019-12-29 23:45:00,Fair
2019-12-29 23:50:00,Fair


## Unindo as duas bases

In [ ]:
df = data.copy()